Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## House Prices Prediction

In this tutorial we will prepare a dataset with houses characteristics and selling prices and train a regression model for sales price prediction.

The dataset to be used is the [Ames Housing Dataset](https://www.openintro.org/stat/data/?data=ames), which has variables describing (almost) every aspect of residential homes in Ames, Iowa.

A detailed description of the variables in this dataset can be found [here](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt).

We begin by importing the necessary packages and setting some notebook options.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

warnings.filterwarnings('ignore')

%matplotlib inline

pd.options.display.max_rows = None
pd.options.display.max_columns = None

Here we download the dataset described above.

In [ ]:
data_folder = "./data"
os.makedirs(data_folder, exist_ok = True)

!wget https://www.openintro.org/stat/data/ames.csv -O ./data/ames.csv

We then load the dataset into a Pandas data frame, visualize the first 10 rows, and print the total number of rows and columns. We notice that this dataset has 2930 rows and 82 columns. Our response variable is the column named `SalePrice`.

In [ ]:
df_housing = pd.read_csv("./data/ames.csv")

df_housing.head(10)

In [ ]:
df_housing.shape

We describe all columns and notice several things:
  - the majority of the variables are categorical
  - some categorical variables are wrongly encoded as numeric
  - some numeric variables are wrongly encoded as categorical
  - there are several missing values

In [ ]:
df_housing.describe(include = "all")

We drop the `Order` and `PID` columns because they are unique identifiers and won't help predicting the house price.

In [ ]:
df_housing["Order"].nunique()

In [ ]:
df_housing["PID"].nunique()

In [ ]:
df_housing.drop("Order", axis = 1, inplace = True)
df_housing.drop("PID", axis = 1, inplace = True)

We begin by treating the missing values. To better analyze this,we create a function that builds a table with the missing percentage for each variable that has missing values.

In [ ]:
def compute_missing_ratio(df):
    df_na = (df.isnull().sum() / len(df)) * 100
    df_na = df_na.drop(df_na[df_na == 0].index).sort_values(ascending = False)
    display(pd.DataFrame({'Missing Ratio' :df_na}))

In [ ]:
compute_missing_ratio(df_housing)


Here we apply some strategies for inputting missing values, according to some hints we can get from the dataset description. 

For example, for some categorical variables a missing value represents a category like "None", and for some numerical variables it represents the value 0.

For variables with relatively few missing values we can perform basic inputations like the median value for numeric variables and the mode value for categorical variables.

In [ ]:
fill_none = ["Pool.QC", "Misc.Feature", "Alley", "Fence", "Fireplace.Qu", 
             "Garage.Type", "Garage.Finish", "Garage.Qual", "Garage.Cond",
            "Bsmt.Exposure", "Bsmt.Cond", "Bsmt.Qual", "Mas.Vnr.Type"]
for var in fill_none:
    df_housing[var] = df_housing[var].fillna("None")
    
fill_zero = ["Garage.Yr.Blt", "BsmtFin.Type.2", "BsmtFin.Type.1", "Bsmt.Half.Bath", 
             "Bsmt.Full.Bath", "Total.Bsmt.SF", "Bsmt.Unf.SF", "BsmtFin.SF.1", 
             "BsmtFin.SF.2", "Garage.Area", "Garage.Cars", "Mas.Vnr.Area"]
for var in fill_zero:
    df_housing[var] = df_housing[var].fillna(0)

df_housing["Lot.Frontage"] = df_housing.groupby("Neighborhood")["Lot.Frontage"].transform(lambda x: x.fillna(x.median()))

df_housing['Electrical'] = df_housing['Electrical'].fillna(df_housing['Electrical'].mode()[0])

In [ ]:
compute_missing_ratio(df_housing)

In [ ]:
df_housing.loc[df_housing["Lot.Frontage"].isnull(), :]

We have inputed missing values for `Lot.Frontage` with the median values of `Lot.Frontage` by `Neighborhood`, but there are still missing values for that variable.

This is because there is one neighborhood with only one house and its `Lot.Frontage` value is missing. And there is another neighborhood with only two houses with both values for `Lot.Frontage` also missing. 

We will then discard those records.

In [ ]:
df_housing = df_housing.dropna()

compute_missing_ratio(df_housing)

Now we correct some data types, according to our interpretation of continuous and categorical variables in this dataset. We represent numerical continuous values as float numbers and categorical as strings.

In [ ]:
df_housing.columns

In [ ]:
response_var = ["SalePrice"]

numeric_vars = ["Lot.Frontage", "Lot.Area", "Mas.Vnr.Area", "BsmtFin.SF.1", "BsmtFin.SF.2", 
                "Bsmt.Unf.SF", "Total.Bsmt.SF", "X1st.Flr.SF", "X2nd.Flr.SF", "Low.Qual.Fin.SF", 
                "Gr.Liv.Area", "Garage.Area", "Wood.Deck.SF", "Open.Porch.SF", "Enclosed.Porch", 
                "X3Ssn.Porch", "Screen.Porch", "Pool.Area", "Misc.Val"]

categorical_vars = [v for v in df_housing.columns if v not in numeric_vars + response_var]

df_housing[response_var] = df_housing[response_var].astype(float)
df_housing[numeric_vars] = df_housing[numeric_vars].astype(float)
df_housing[categorical_vars] = df_housing[categorical_vars].astype(str)

display(pd.DataFrame({"Data Type": df_housing.dtypes}))

After finishing the data cleaning, we then visualize relashionships between variables.

We begin with scatterplots between `SalePrice` and continuous variables.

In [ ]:
sns.pairplot(df_housing, y_vars=response_var, x_vars=numeric_vars[0:7])
sns.pairplot(df_housing, y_vars=response_var, x_vars=numeric_vars[7:13])
sns.pairplot(df_housing, y_vars=response_var, x_vars=numeric_vars[13:19])

Now we create boxplots of `SalesPrice` according to the categories given by the categorical variables.

To better visualize this, we first encode each categorical variable by ordering its distinct category values according to the mean of `SalePrice` calculated for each category value.

In [ ]:
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0] + 1)
    ordering = ordering['ordering'].to_dict()
    
    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature + '_E'] = o
    
categorical_vars_E = []
for q in categorical_vars:  
    encode(df_housing, q)
    categorical_vars_E.append(q + '_E')

def boxplot(x, y, **kwargs):
    sns.boxplot(x = x, y = y)
    x = plt.xticks(rotation = 90)

data = pd.melt(df_housing, id_vars = ["SalePrice"], value_vars = categorical_vars_E)
g = sns.FacetGrid(data, col = "variable",  col_wrap = 5, sharex = False, sharey = False)
g = g.map(boxplot, "value", "SalePrice")

Here we plot the Spearman Correlation between `SalePrice` and each variable.

For this to make sense for the categorical variables, we use the previous numeric ordered encoded values to represent each of them.

In [ ]:
def spearman(frame, features):
    spr = pd.DataFrame()
    spr['feature'] = features
    spr['spearman'] = [frame[f].corr(frame['SalePrice'], 'spearman') for f in features]
    spr = spr.sort_values('spearman')
    plt.figure(figsize = (6, 0.25*len(features)))
    sns.barplot(data = spr, y = 'feature', x = 'spearman', orient = 'h')
    
features = numeric_vars + categorical_vars_E
spearman(df_housing, features)

Now we visualize the distribution of `SalePrice`.

In [ ]:
sns.distplot(df_housing[response_var])

In [ ]:
sns.boxplot(df_housing[response_var])

We are not going to perform any outlier analysis, feature selection or transformation here. Instead, we will try to model `SalePrice` directly using a non-linear regression algorithm.

As an example, we will use Gradient Boosting Regression.

The first step here is to split the dataset in a training portion and a test portion for final evaluation.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_housing[numeric_vars + categorical_vars_E], 
                                                    df_housing[response_var],
                                                    test_size = 0.4, random_state = 0)

Now we create a loop for model training with grid search for hyperparameter selection and using the training data for 5-fold cross-validation.

In [ ]:
parameter_grid = [{'n_estimators': [250,500,1000], 'max_depth': [4,8], 'min_samples_split': [2,4],
                   'learning_rate': [0.01], 'loss': ['ls']}]

scores = {'R2': make_scorer(r2_score), 'MAE': make_scorer(mean_absolute_error)}

cv_models = {}

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(GradientBoostingRegressor(), parameter_grid, cv = 10, scoring = scores[score], n_jobs = -1)
    clf.fit(X_train, y_train)
    cv_models[score] = clf

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

Get the best model and compute metrics for the test dataset.

In [ ]:
best_model = cv_models['R2'].best_estimator_

mae = mean_absolute_error(y_test, best_model.predict(X_test))
print("MAE on test dataset: %.4f" % mae)

y_test_predicted = best_model.predict(X_test)
r2 = r2_score(y_test, y_test_predicted)
print("R2 on test dataset: %.4f" % r2)

Compute relative feature importances using the best model.

In [ ]:
feature_importance = best_model.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())

sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
var_names = np.asarray(numeric_vars + categorical_vars)
fig = plt.figure(figsize=(12, 20))
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, var_names[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [ ]:
from sklearn.model_selection import cross_validate

scores = {'R2': make_scorer(r2_score), 'MAE': make_scorer(mean_absolute_error)}

clf = GradientBoostingRegressor(n_estimators = 500, max_depth = 4, min_samples_split = 2, 
                                learning_rate = 0.01, loss = "ls")
scores = cross_validate(clf, X_train, y_train, scoring = scores, cv = 5)

In [ ]:
scores

In [ ]:
clf_fitted = clf.fit(X_train, y_train)
y_test_predicted = clf_fitted.predict(X_test)

mae = mean_absolute_error(y_test, y_test_predicted)
print("MAE on test dataset: %.4f" % mae)

r2 = r2_score(y_test, y_test_predicted)
print("R2 on test dataset: %.4f" % r2)

In [ ]:
scores["test_R2"].mean()